# Install torchmin
Installing torchmin with
```
pip install pytorch-minimize
```
Documentation for [torchmin](https://pytorch-minimize.readthedocs.io/en/latest/index.html)

# Using torchmin for constrained optimization
```
minimize_constr(f, x0[, constr, bounds, …])
```



In [46]:
import torch
#import torch.nn as nn
import numpy as np
import matplotlib.pylab as plt

from torchmin import minimize_constr

torch.manual_seed(382)
pars0 = torch.randn(2, requires_grad=True)
pars0 = pars0.square()
#pars0 = torch.Variable(pars0, requires_grad=True)
pars = torch.nn.Parameter(pars0, requires_grad =True)
# pars  = pars0

opt = torch.optim.SGD(params=[pars], lr=0.1)


def loss_function(pars):
    return pars[0]*pars[1]

print(pars[0], pars0, loss_function(pars0))
# res = minimize_constr(
#     loss_function, pars0, 
#     max_iter=100,
#     constr=dict(
#         fun=lambda x: x[0]+x[1], 
#         lb=1, ub=1
#     ),
#     bounds=dict(
#         lb=0, ub=1
#     ),
#     disp=1
# )

def loss_function2(p):
    return p*(p-1)
# p0 = torch.randn(1, requires_grad=True)
# p = torch.nn.Parameter(p0, requires_grad =True)
# res = minimize_constr(
#     loss_function2, p, 
#     max_iter=100,
#     constr=dict(
#         fun=lambda x: x, 
#         lb=1, ub=1
#     ),
#     disp=1
# )
x0=torch.tensor(0.6)
def constr1(pars):
    return pars.sum()
# res=minimize_constr(loss_function2, x0,constr=dict(fun=constr1, lb=1, ub=1),bounds=dict(lb=0,ub=1),tol=1e-9,disp=1)
#res=minimize_constr(loss_function,pars0,constr=dict(fun=constr1, lb=1, ub=1),bounds=dict(lb=0,ub=1),tol=1e-9,disp=1)
print(pars0)
#print(pars0.square())
# res=minimize_constr(loss_function,pars0,constr=dict(fun=lambda x: x.sum(), lb=1, ub=1),bounds=dict(lb=0,ub=1),tol=1e-9,disp=1)
res=minimize_constr(loss_function,pars0,constr=dict(fun=lambda x: x.sum(), lb=1, ub=1),bounds=dict(lb=0,ub=1),tol=1e-9,disp=1)

for epoch in range(100):
    opt.zero_grad()
    loss = loss_function(pars)
    loss.backward()
    print(pars.grad)
    opt.step()
    print(pars)
    print(res.x,res.fun)
    pq = [par.item() for par in pars[:2]]
    plt.plot(pq[0], pq[1], ".")

# pars = pars.detach()
# pars[7:].clamp_(0)
print(pars)

plt.plot(np.linspace(0,1), np.ones(50)-np.linspace(0,1))
plt.show()


tensor(0.2638, grad_fn=<SelectBackward0>) tensor([0.2638, 0.4921], grad_fn=<PowBackward0>) tensor(0.1298, grad_fn=<MulBackward0>)
tensor([0.2638, 0.4921], grad_fn=<PowBackward0>)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
class constraint_sum_to_1(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)